# Gaussian processes (2/3) - Fitting a Gaussian process kernel

In the [previous post]({% post_url /blog/gaussian_process/2019-01-05-gaussian-process-tutorial %}) we introduced the Gaussian process model with the exponentiated quadratic covariance function. In this post we will introduce parametrized covariance functions (kernels), fit them to real world data, and use them to make posterior predictions. This post is the second part of a series on Gaussian processes:

1. [Understanding Gaussian processes]({% post_url /blog/gaussian_process/2019-01-05-gaussian-process-tutorial %})
2. [Fitting a Gaussian process kernel (this)]({% post_url /blog/gaussian_process/2019-01-06-gaussian-process-kernel-fitting %})
3. [Gaussian process kernels]({% post_url /blog/gaussian_process/2019-01-07-gaussian-process-kernels %})

We will implement the Gaussian process model in [TensorFlow Probability](https://www.tensorflow.org/probability/) which will allow us to easily implement and tune our model without having to worry about the details.

In [1]:
# Imports
import os
from itertools import islice
import warnings
warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp

from tqdm.notebook import tqdm
import bokeh
import bokeh.io
import bokeh.plotting
import bokeh.models
from IPython.display import display, HTML

bokeh.io.output_notebook(hide_banner=True)

tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels

np.random.seed(42)
tf.random.set_seed(42)
#

## Mauna Loa CO₂ data

The dataset used in this example is the monthly average [atmospheric CO₂](https://en.wikipedia.org/wiki/Carbon_dioxide_in_Earth%27s_atmosphere) concentrations (in parts per million (ppm)) collected at the [Mauna Loa Observatory](https://en.wikipedia.org/wiki/Mauna_Loa_Observatory) in Hawaii. The observatory has been collecting these CO₂ concentrations since 1958 and [showed](https://en.wikipedia.org/wiki/Keeling_Curve) the first significant evidence of rapidly increasing CO₂ levels in the atmosphere.

These measures of atmospheric CO₂ concentrations show different characteristics such as a long term rising trend, variation with the seasons, and smaller irregularities. This made it into a canonical example in Gaussian process modelling[⁽¹⁾](#References).

In this post the data is downloaded as a CSV file from the [Scripps CO₂ Program website](https://scrippsco2.ucsd.edu/data/atmospheric_co2/primary_mlo_co2_record.html). This data is loaded in a pandas dataframe and plotted below.

In [10]:
# Load the data
# Load the data from the Scripps CO2 program website.
co2_df = pd.read_csv(
    'https://scrippsco2.ucsd.edu/assets/data/atmospheric/stations/in_situ_co2/monthly/monthly_in_situ_co2_mlo.csv',
    # './monthly_in_situ_co2_mlo.csv',
    header=61, # Data starts here
    skiprows=[62, 63], # Headers consist of multiple rows
    usecols=[3, 4], # Only keep the 'Date' and 'CO2' columns
    na_values='-99.99',  # NaNs are denoted as '-99.99'
    dtype=np.float64
)

# Drop missing values
co2_df.dropna(inplace=True)
# Remove whitespace from column names
co2_df.rename(columns=lambda x: x.strip(), inplace=True)
#
print(co2_df)

          Date     CO2
2    1958.2027  315.71
3    1958.2877  317.45
4    1958.3699  317.51
6    1958.5370  315.87
7    1958.6219  314.93
..         ...     ...
797  2024.4563  426.62
798  2024.5383  425.40
799  2024.6230  422.70
800  2024.7077  421.59
801  2024.7896  422.05

[795 rows x 2 columns]


In [11]:
# Plot data
fig = bokeh.plotting.figure(
    width=600, height=300,
    x_range=(1958, 2021), y_range=(310, 420))
fig.xaxis.axis_label = 'Date'
fig.yaxis.axis_label = 'CO₂ (ppm)'
fig.add_layout(bokeh.models.Title(
    text='In situ air measurements at Mauna Loa, Observatory, Hawaii',
    text_font_style="italic"), 'above')
fig.add_layout(bokeh.models.Title(
    text='Atmospheric CO₂ concentrations',
    text_font_size="14pt"), 'above')
fig.line(
    co2_df.Date, co2_df.CO2, legend_label='All data',
    line_width=2, line_color='midnightblue')
fig.legend.location = 'top_left'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)
#

In [12]:
# Split the data into observed and to predict
date_split_predict = 2010
df_observed = co2_df[co2_df.Date < date_split_predict]
print('{} measurements in the observed set'.format(len(df_observed)))
df_predict = co2_df[co2_df.Date >= date_split_predict]
print('{} measurements in the test set'.format(len(df_predict)))
#

617 measurements in the observed set
178 measurements in the test set


## Gaussian process model

We're going to use a Gaussian process model to make posterior predictions of the atmospheric CO₂ concentrations for 2010 and after based on the observed data from before 2010.

A Gaussian process is uniquely defined by it's mean function $m(x)$ and covariance function $k(x,x')$:

$$f(x) \sim \mathcal{GP}(m(x),k(x,x'))$$

### Mean function

Since most interesting effects will be modelled by the kernel function we will keep the mean function simple. In this example the mean function is going to be modelled as a function that always returns the mean of the observations.

In [13]:
# Define mean function which is the means of observations
observations_mean = tf.constant(
    [np.mean(df_observed.CO2.values)], dtype=tf.float64)
mean_fn = lambda _: observations_mean
#

### Kernel function

To model the different characteristics of our dataset we will create a covariance (kernel) function by combining different kernels already available in [TensorFlow-Probability](https://www.tensorflow.org/probability/api_docs/python/tfp/math/psd_kernels/PositiveSemidefiniteKernel). The different data characterics will be modelled as:

- Long term smooth change in CO₂ levels over time modelled by a [exponentiated quadratic](https://www.tensorflow.org/probability/api_docs/python/tfp/math/psd_kernels/ExponentiatedQuadratic) kernel defined in the code below as `smooth_kernel`.
- Seasonality based on a local periodic kernel, which consists of a [exponentiated sine squared kernel](https://www.tensorflow.org/probability/api_docs/python/tfp/math/psd_kernels/ExpSinSquared) multiplied with a exponentiated quadratic to make the seasonality degrade as further it gets from the observations. This seasonal periodic kernel is defined in the code below as `local_periodic_kernel`.
- Short to medium term irregularities modelled by a rational quadratic kernel, which is defined in the code below as `irregular_kernel`.
- Observational noise which will be modelled directly by the `observation_noise_variance` parameters of the [TensorFlow Gaussian process model](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/GaussianProcess).

These different kernels will be summed into one single kernel function $k_{\theta}(x_a, x_b)$ that will allow for all these effects to occur together. This kernel is defined as `kernel` in the code below. Each of the kernels has hyperparameters $\theta$ that can be tuned, they will be defined as a variable so they can be fitted on the observed data. [This post]({% post_url /blog/gaussian_process/2019-01-07-gaussian-process-kernels %}) provides more insight on the kernels used here and the effect of their hyperparameters.

In [17]:
# Define the kernel with trainable parameters.
# Note we transform some of the trainable variables to ensure
#  they stay positive.

# Use float64 because this means that the kernel matrix will have
#  less numerical issues when computing the Cholesky decomposition

# Constrain to make sure certain parameters are strictly positive
constrain_positive = tfb.Shift(np.finfo(np.float64).tiny)(tfb.Exp())

# Smooth kernel hyperparameters
smooth_amplitude = tfp.util.TransformedVariable(
    initial_value=10., bijector=constrain_positive, dtype=np.float64,
    name='smooth_amplitude')
smooth_length_scale = tfp.util.TransformedVariable(
    initial_value=10., bijector=constrain_positive, dtype=np.float64,
    name='smooth_length_scale')
# Smooth kernel
smooth_kernel = tfk.ExponentiatedQuadratic(
    amplitude=smooth_amplitude,
    length_scale=smooth_length_scale)

# Local periodic kernel hyperparameters
periodic_amplitude = tfp.util.TransformedVariable(
    initial_value=5.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_amplitude')
periodic_length_scale = tfp.util.TransformedVariable(
    initial_value=1.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_length_scale')
periodic_period = tfp.util.TransformedVariable(
    initial_value=1.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_period')
periodic_local_length_scale = tfp.util.TransformedVariable(
    initial_value=1.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_local_length_scale')
# Local periodic kernel
local_periodic_kernel = (
    tfk.ExpSinSquared(
        amplitude=periodic_amplitude,
        length_scale=periodic_length_scale,
        period=periodic_period) *
    tfk.ExponentiatedQuadratic(
        length_scale=periodic_local_length_scale))

# Short-medium term irregularities kernel hyperparameters
irregular_amplitude = tfp.util.TransformedVariable(
    initial_value=1., bijector=constrain_positive, dtype=np.float64,
    name='irregular_amplitude')
irregular_length_scale = tfp.util.TransformedVariable(
    initial_value=1., bijector=constrain_positive, dtype=np.float64,
    name='irregular_length_scale')
irregular_scale_mixture = tfp.util.TransformedVariable(
    initial_value=1., bijector=constrain_positive, dtype=np.float64,
    name='irregular_scale_mixture')
# Short-medium term irregularities kernel
irregular_kernel = tfk.RationalQuadratic(
    amplitude=irregular_amplitude,
    length_scale=irregular_length_scale,
    scale_mixture_rate=irregular_scale_mixture)

# Noise variance of observations
# Start out with a medium-to high noise
observation_noise_variance = tfp.util.TransformedVariable(
    initial_value=1, bijector=constrain_positive, dtype=np.float64,
    name='observation_noise_variance')

trainable_variables = [v.variables[0] for v in [
    smooth_amplitude,
    smooth_length_scale,
    periodic_amplitude,
    periodic_length_scale,
    periodic_period,
    periodic_local_length_scale,
    irregular_amplitude,
    irregular_length_scale,
    irregular_scale_mixture,
    observation_noise_variance
]]

#

In [18]:
# Sum all kernels to single kernel containing all characteristics
kernel = (smooth_kernel + local_periodic_kernel + irregular_kernel)

### Tuning the hyperparameters

We can tune the hyperparameters $\theta$ of our Gaussian process model based on the data. This post leverages TensorFlow to fit the parameters by maximizing the marginal likelihood  $p(\mathbf{y} \mid X, \theta)$ of the Gaussian process distribution based on the observed data $(X, \mathbf{y})$.

$$\hat{\theta}  = \underset{\theta}{\text{argmax}} \left( p(\mathbf{y} \mid X, \theta) \right)$$

The marginal likelihood of the Gaussian process is the likelihood of a [Gaussian distribution]({% post_url /blog/misc/2018-09-28-multivariate-normal-primer %}) which is defined as:

$$p(\mathbf{y} \mid \mu, \Sigma) = \frac{1}{\sqrt{(2\pi)^d \lvert\Sigma\rvert}} \exp{ \left( -\frac{1}{2}(\mathbf{y} - \mu)^{\top} \Sigma^{-1} (\mathbf{y} - \mu) \right)}$$

The mean and covariance are calculated from their parameterized functions using the observed data $X$ as input: $\mu_{\theta} = m_{\theta}(X)$ and $\Sigma_{\theta} = k_{\theta}(X, X)$, so we can write the marginal likelihood as:

$$
p(\mathbf{y} \mid X, \theta) =
\frac{1}{\sqrt{(2\pi)^d \lvert \Sigma_{\theta} \rvert}} \exp{ \left( -\frac{1}{2}(\mathbf{y} - \mu_{\theta})^\top \Sigma_{\theta}^{-1} (\mathbf{y} - \mu_{\theta}) \right)}$$

With $d$ the dimensionality of the marginal and $\lvert \Sigma_{\theta} \rvert$ the determinant of the kernel matrix. We can get rid of the exponent by taking the log and maximizing the log marginal likelihood:

$$ \log{p(\mathbf{y} \mid X, \theta)} =
-\frac{1}{2}(\mathbf{y} - \mu_{\theta})^\top \Sigma_{\theta}^{-1} (\mathbf{y} - \mu_{\theta}) - \frac{1}{2} \log{\lvert \Sigma_{\theta} \rvert} - \frac{d}{2} \log{2 \pi}$$

The first term ($-0.5 (\mathbf{y} - \mu_{\theta})^\top \Sigma_{\theta}^{-1} (\mathbf{y} - \mu_{\theta})$) is the data-fit while the rest ($-0.5(\log{\lvert \Sigma_{\theta} \rvert} + d\log{2 \pi})$) is a complexity penalty, also known as [differential entropy](https://en.wikipedia.org/wiki/Differential_entropy)[⁽¹⁾](#Sidenotes).



The optimal parameters $\hat{\theta}$ can then be found by minimizing the negative of the log marginal likelihood:

$$
\hat{\theta}  = \underset{\theta}{\text{argmax}} \left( p(\mathbf{y} \mid X, \theta) \right) = \underset{\theta}{\text{argmin}} { \;-\log{ p(\mathbf{y} \mid X, \theta)}}$$

Since in this case the log marginal likelihood is derivable with respect to the kernel hyperparameters, we can use a [gradient based approach]({% post_url /blog/neural_net_implementation/2015-06-12-neural-network-implementation-part01 %}) to minimize the negative log marginal likelihood (NLL). In this post we will be using a gradient descent based approach to train the hyperparameters on minibatches of the observed data.

In [14]:
# Define mini-batch data iterator
batch_size = 128

batched_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (df_observed.Date.values.reshape(-1, 1), df_observed.CO2.values))
    .shuffle(buffer_size=len(df_observed))
    .repeat(count=None)
    .batch(batch_size)
)
#

We will represent the Gaussian process marginal distribution by the [`GaussianProcess`](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/GaussianProcess) object which has a `log_prob` function to get the marginal log likelihood. The [`AdamOptimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) is used to minimize the negative marginal log likelihood.

In [15]:
# Use tf.function for more efficient function evaluation
@tf.function(autograph=False, experimental_compile=False)
def gp_loss_fn(index_points, observations):
    """Gaussian process negative-log-likelihood loss function."""
    gp = tfd.GaussianProcess(
        mean_fn=mean_fn,
        kernel=kernel,
        index_points=index_points,
        observation_noise_variance=observation_noise_variance
    )

    negative_log_likelihood = -gp.log_prob(observations)
    return negative_log_likelihood

In [19]:
# Fit hyperparameters
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Training loop
batch_nlls = []  # Batch NLL for plotting
full_ll = []  # Full data NLL for plotting
nb_iterations = 10001
for i, (index_points_batch, observations_batch) in tqdm(
        enumerate(islice(batched_dataset, nb_iterations)), total=nb_iterations):
    # Run optimization for single batch
    with tf.GradientTape() as tape:
        loss = gp_loss_fn(index_points_batch, observations_batch)
    grads = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(grads, trainable_variables))
    batch_nlls.append((i, loss.numpy()))
    # Evaluate on all observations
    if i % 100 == 0:
        # Evaluate on all observed data
        ll = gp_loss_fn(
            index_points=df_observed.Date.values.reshape(-1, 1),
            observations=df_observed.CO2.values)
        full_ll.append((i, ll.numpy()))
#

  0%|          | 0/10001 [00:00<?, ?it/s]

Instructions for updating:
`jitter` is deprecated; please use `marginal_fn` directly.


In [20]:
# Plot NLL over iterations
fig = bokeh.plotting.figure(
    width=600, height=350,
    x_range=(0, nb_iterations), y_range=(50, 200))
fig.add_layout(bokeh.models.Title(
    text='Negative Log-Likelihood (NLL) during training',
    text_font_size="14pt"), 'above')
fig.xaxis.axis_label = 'iteration'
fig.yaxis.axis_label = 'NLL batch'
# First plot
fig.line(
    *zip(*batch_nlls), legend_label='Batch data',
    line_width=2, line_color='midnightblue')
# Seoncd plot
# Setting the second y axis range name and range
fig.extra_y_ranges = {
    'fig1ax2': bokeh.models.Range1d(start=130, end=250)}
fig.line(
    *zip(*full_ll), legend_label='All observed data',
    line_width=2, line_color='red', y_range_name='fig1ax2')
# Adding the second axis to the plot.
fig.add_layout(bokeh.models.LinearAxis(
    y_range_name='fig1ax2', axis_label='NLL all'), 'right')

fig.legend.location = 'top_right'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)
#

Despite the complexity term in the log marginal likelihood it is still possible to overfit to the data. While we didn't prevent further overfitting in this post, you could prevent overfitting by adding a regularization term on the hyperparameters[⁽²⁾](#Sidenotes), or split the observations in training an validation sets and select the best fit on your validation data from models trained on the training data.

In [21]:
# Show values of parameters found
variables = [
    smooth_amplitude,
    smooth_length_scale,
    periodic_amplitude,
    periodic_length_scale,
    periodic_period,
    periodic_local_length_scale,
    irregular_amplitude,
    irregular_length_scale,
    irregular_scale_mixture,
    observation_noise_variance
]

data = list([(var.variables[0].name[:-2], var.numpy()) for var in variables])
df_variables = pd.DataFrame(
    data, columns=['Hyperparameters', 'Value'])
display(HTML(df_variables.to_html(
    index=False, float_format=lambda x: f'{x:.4f}')))
#

Hyperparameters,Value
smooth_amplitude,107.45833656801197
smooth_length_scale,90.17528751008628
periodic_amplitude,3.097324217249989
periodic_length_scale,1.6771144650202505
periodic_period,0.9996041569333632
periodic_local_length_scale,105.63879643150774
irregular_amplitude,0.9945908613291353
irregular_length_scale,1.3793759423759664
irregular_scale_mixture,0.11139412371272481
observation_noise_variance,0.05366403847660918


The fitted kernel and it's components are illustrated in more detail in a [follow-up post]({% post_url /blog/gaussian_process/2019-01-07-gaussian-process-kernels %}#Atmospheric-CO%E2%82%82-kernel).

### Posterior predictions

The TensorFlow `GaussianProcess` class can only represent an unconditional Gaussian process.
To make predictions by posterior inference conditioned on observed data we will need to create a [`GaussianProcessRegressionModel`](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/GaussianProcessRegressionModel) with the fitted kernel, mean function and observed data.

The posterior predictions conditioned on the observed data before 2010 are plotted in the figure below together with their 95% prediction interval.

Notice that our model captures captures the different dataset characteristics such as the trend and seasonality quite well. The predictions start deviating the further away from the observed data the model was conditioned on, together with widening prediction interval.

In [ ]:
# Posterior GP using fitted kernel and observed data
gp_posterior_predict = tfd.GaussianProcessRegressionModel(
    mean_fn=mean_fn,
    kernel=kernel,
    index_points=df_predict.Date.values.reshape(-1, 1),
    observation_index_points=df_observed.Date.values.reshape(-1, 1),
    observations=df_observed.CO2.values,
    observation_noise_variance=observation_noise_variance)

# Posterior mean and standard deviation
posterior_mean_predict = gp_posterior_predict.mean()
posterior_std_predict = gp_posterior_predict.stddev()

In [ ]:
# Plot posterior predictions

# Get posterior predictions
μ = posterior_mean_predict.numpy()
σ = posterior_std_predict.numpy()

# Plot
fig = bokeh.plotting.figure(
    width=600, height=400,
    x_range=(2010, 2021.3), y_range=(384, 418))
fig.xaxis.axis_label = 'Date'
fig.yaxis.axis_label = 'CO₂ (ppm)'
fig.add_layout(bokeh.models.Title(
    text='Posterior predictions conditioned on observations before 2010.',
    text_font_style="italic"), 'above')
fig.add_layout(bokeh.models.Title(
    text='Atmospheric CO₂ concentrations',
    text_font_size="14pt"), 'above')
fig.circle(
    co2_df.Date, co2_df.CO2, legend_label='True data',
    size=2, line_color='midnightblue')
fig.line(
    df_predict.Date.values, μ, legend_label='μ (predictions)',
    line_width=2, line_color='firebrick')
# Prediction interval
band_x = np.append(
    df_predict.Date.values, df_predict.Date.values[::-1])
band_y = np.append(
    (μ + 2*σ), (μ - 2*σ)[::-1])
fig.patch(
    band_x, band_y, color='firebrick', alpha=0.4,
    line_color='firebrick', legend_label='2σ')

fig.legend.location = 'top_left'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)
#

This post illustrated using TensorFlow probability to combine multiple out-of-the-box kernels and fit their hyperparameters on observed data. The fitted model was then used to make posterior predictions.

This post was the second part of a series on Gaussian processes:

1. [Understanding Gaussian processes]({% post_url /blog/gaussian_process/2019-01-05-gaussian-process-tutorial %})
2. [Fitting a Gaussian process kernel (this)]({% post_url /blog/gaussian_process/2019-01-06-gaussian-process-kernel-fitting %})
3. [Gaussian process kernels]({% post_url /blog/gaussian_process/2019-01-07-gaussian-process-kernels %})

## Sidenotes

1. Note that the [determinant](https://en.wikipedia.org/wiki/Determinant) $\lvert \Sigma \rvert$ is equal to the product of it's [eigenvalues](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors), and that $\lvert \Sigma \rvert$ can be interpreted as the [volume spanned by](https://en.wikipedia.org/wiki/Determinant#Volume_and_Jacobian_determinant) the covariance matrix $\Sigma$. Reducing $\lvert \Sigma \rvert$ will thus decrease the dispersion of the points coming from the distribution with covariance matrix $\Sigma$ and reduce the complexity.
2. Using TensorFlow Probability [distributions](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions) you can define a prior distribution for each hyperparameter and use the log-likelihood of this distribution as a regularization term on your parameter.

## References

1. [Gaussian Processes for Machine Learning. Chapter 5: Model Selection and Adaptation of Hyperparameters](http://www.gaussianprocess.org/gpml/chapters/RW5.pdf) by Carl Edward Rasmussen and Christopher K. I. Williams.
2. [Gaussian Processes for Regression](https://papers.nips.cc/paper/1048-gaussian-processes-for-regression.pdf) by Christopher K. I. Williams and Carl Edward Rasmussen.

In [ ]:
# Python package versions used
%load_ext watermark
%watermark --python
%watermark --iversions
#

Python implementation: CPython
Python version       : 3.9.4
IPython version      : 7.23.1

bokeh                 : 2.3.2
pandas                : 1.2.4
tensorflow_probability: 0.12.2
tensorflow            : 2.5.0
numpy                 : 1.19.5



This post at <a rel="canonical" href="https://peterroelants.github.io/posts/gaussian-process-kernel-fitting/">peterroelants.github.io</a> is generated from an Python notebook file. [Link to the full IPython notebook file](https://github.com/peterroelants/peterroelants.github.io/blob/main/notebooks/gaussian_process/gaussian-process-kernel-fitting.ipynb)